In [2]:
## checking gpus

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Wed Nov 30 21:35:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
### Importing necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
#import torchviz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import torch.nn.functional as F
import torch.optim as optim
# %matplotlib notebook
%matplotlib inline
from tqdm.notebook import tqdm
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
pd.set_option("display.max_columns", 50, "display.max_rows", 100)

In [5]:
# importing required modules
# final_project/jsons_data.csv.zip
# /content/jsons_data.csv.zip
# /jsons_data.csv.zip
#from zipfile import ZipFile
from zipfile import ZipFile 
#/content/jsons_data.csv.zip
  
# specifying the zip file name
file_name = "/content/jsons_data.csv.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as new:
    # printing all the contents of the zip file
    new.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    new.extractall()
    print('Done!')

File Name                                             Modified             Size
jsons_data.csv                                 2022-11-24 13:40:00     65882315
__MACOSX/._jsons_data.csv                      2022-11-24 13:40:00          233
Extracting all the files now...
Done!


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
# final_project/jsons_data.csv
df = pd.read_csv("/content/jsons_data.csv")
df.head()

,Unnamed: 0,abstract,title,keywords
0,0,Glioblastoma is the most common and most aggre...,PEITC induces apoptosis of Human Brain Gliobla...,"['Apoptosis', 'Caspases', 'Glioblastoma', 'Mit..."
1,1,Chiral amino alcohols are structural motifs pr...,Multi-step biocatalytic strategies for chiral ...,"['Cascades', 'Chiral amino alcohols', 'Recycli..."
2,2,A new dengue vaccine was associated with incre...,Dengue vaccine safety signal: Immune enhanceme...,"['Adverse events following immunization', 'Cli..."
3,3,This paper explores the tension between post-f...,Achieving respectable motherhood? Exploring th...,NaN
4,4,A key strategy to enable training of deep neur...,"Soft++, a multi-parametric non-saturating non-...","['Activation function', 'Convolutional neural ..."


In [ ]:
df = df[["abstract", "title"]]
df = df.applymap(str)
# only preserve needed columns
df.head()

,abstract,title
0,Glioblastoma is the most common and most aggre...,PEITC induces apoptosis of Human Brain Gliobla...
1,Chiral amino alcohols are structural motifs pr...,Multi-step biocatalytic strategies for chiral ...
2,A new dengue vaccine was associated with incre...,Dengue vaccine safety signal: Immune enhanceme...
3,This paper explores the tension between post-f...,Achieving respectable motherhood? Exploring th...
4,A key strategy to enable training of deep neur...,"Soft++, a multi-parametric non-saturating non-..."


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle

In [ ]:
df.shape

(40001, 2)

In [ ]:
document = df['abstract']
summary = df['title']

In [ ]:
document[30], summary[30]

('This paper presents GIS time-series land-use analysis of satellite images to quantify the recovery of rice cultivation and aquaculture following the 2004 Indian Ocean tsunami in coastal communities in Aceh, Indonesia. We supplement this with qualitative data to illustrate the post-disaster challenges faced by residents, and the extent to which coastal communities have adapted to post-tsunami realities. Our analysis shows that the rehabilitation of rice cultivation and aquaculture in areas inundated by the tsunami has been limited by extensive degradation of land, diversion of labor by tsunami mortality and transition to alternative livelihoods, and re-purposing of rice fields for residential use during the reconstruction phase. This is especially prominent in areas where subsistence activities are not the primary source of livelihood. The Aceh case study shows that social, economic, and environmental factors can be stronger determinants of how coastal livelihoods rebound and change f

## Preprocessing

In [ ]:
# for decoder sequence
summary = summary.apply(lambda x: ' ' + x + ' ')
summary.head()

0         PEITC induces apoptosis of Human Brain Gl...
1         Multi-step biocatalytic strategies for ch...
2         Dengue vaccine safety signal: Immune enha...
3         Achieving respectable motherhood? Explori...
4         Soft++, a multi-parametric non-saturating...
Name: title, dtype: object

## Tokenizing the texts into integer tokens

In [ ]:
# since < and > from default tokens cannot be removed
filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'
oov_token = ''

In [ ]:
document_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token=oov_token)
summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters=filters, oov_token=oov_token)

In [ ]:
document_tokenizer.fit_on_texts(document)
summary_tokenizer.fit_on_texts(summary)

In [ ]:
inputs = document_tokenizer.texts_to_sequences(document)
targets = summary_tokenizer.texts_to_sequences(summary)

In [ ]:
summary_tokenizer.texts_to_sequences(["This is a test"])

[[4936, 33, 6, 409]]

In [ ]:
summary_tokenizer.sequences_to_texts([[4936, 33, 6, 409]])

['this is a test']

In [ ]:
encoder_vocab_size = len(document_tokenizer.word_index) + 1
decoder_vocab_size = len(summary_tokenizer.word_index) + 1

# vocab_size
encoder_vocab_size, decoder_vocab_size

(149751, 43707)

## Obtaining insights on lengths for defining maxlen

In [ ]:
document_lengths = pd.Series([len(x) for x in document])
summary_lengths = pd.Series([len(x) for x in summary])

In [ ]:
document_lengths.describe()

count    40001.000000
mean      1434.145946
std        555.491389
min          3.000000
25%       1069.000000
50%       1401.000000
75%       1748.000000
max       6993.000000
dtype: float64

In [ ]:
summary_lengths.describe()

count    40001.000000
mean       104.878078
std         35.047157
min         11.000000
25%         80.000000
50%        102.000000
75%        126.000000
max        378.000000
dtype: float64

In [ ]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
#encoder_maxlen = 400
#decoder_maxlen = 75

encoder_maxlen = 1434
decoder_maxlen = 104

## Padding/Truncating sequences for identical sequence lengths

In [ ]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')



## Creating dataset pipeline

In [ ]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional


In [ ]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

Masking
Padding mask for masking "pad" sequences
Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model
Scaled Dot Product

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

## Multi-Headed Attention

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

## Feed Forward Network

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

### Fundamental Unit of Transformer encoder

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

### Fundamental Unit of Transformer decoder

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2

### Encoder consisting of multiple EncoderLayer(s)

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x

### Decoder consisting of multiple DecoderLayer(s)

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights

### Finally, the Transformer

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights

## Training

In [ ]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 5

### Adam optimizer with custom learning rate scheduling

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

### Defining losses and other metrics

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

### Transformer

In [ ]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

### Masks

In [ ]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask

### Checkpoints

In [ ]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


## Training steps

In [ ]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [ ]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.4302
Epoch 1 Batch 429 Loss 2.6339
Epoch 1 Loss 2.7434
Time taken for 1 epoch: 315.0805377960205 secs

Epoch 2 Batch 0 Loss 2.2651
Epoch 2 Batch 429 Loss 2.5821
Epoch 2 Loss 2.6934
Time taken for 1 epoch: 300.2356572151184 secs

Epoch 3 Batch 0 Loss 2.3728
Epoch 3 Batch 429 Loss 2.5319
Epoch 3 Loss 2.6396
Time taken for 1 epoch: 300.21465253829956 secs

Epoch 4 Batch 0 Loss 2.2955
Epoch 4 Batch 429 Loss 2.4854
Epoch 4 Loss 2.5949
Time taken for 1 epoch: 300.2170763015747 secs

Epoch 5 Batch 0 Loss 2.0718
Epoch 5 Batch 429 Loss 2.4359
Saving checkpoint for epoch 5 at checkpoints/ckpt-6
Epoch 5 Loss 2.5455
Time taken for 1 epoch: 300.9813542366028 secs



## Inference

## Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [ ]:
def evaluate(input_document):
    input_document = document_tokenizer.texts_to_sequences([input_document])
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')

    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [summary_tokenizer.word_index[""]]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.word_index[""]:
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [ ]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized[1:], 0)  # not printing  token
    return summary_tokenizer.sequences_to_texts(summarized)[0]  # since there is just one translated document

In [ ]:
summarize(
    "This paper presents GIS time-series land-use analysis of satellite images to quantify \
     the recovery of rice cultivation and aquaculture following the 2004 Indian Ocean tsunami \
      in coastal communities in Aceh, Indonesia. We supplement this with qualitative data to \
       illustrate the post-disaster challenges faced by residents, and the extent to which coastal \
        communities have adapted to post-tsunami realities. Our analysis shows that the rehabilitation \
         of rice cultivation and aquaculture in areas inundated by the tsunami has been limited by \
          extensive degradation of land, diversion of labor by tsunami mortality and transition to \
           alternative livelihoods, and re-purposing of rice fields for residential use during the \
            reconstruction phase. This is especially prominent in areas where subsistence activities \
             are not the primary source of livelihood. The Aceh case study shows that social, economic, and \
              environmental factors can be stronger determinants of how coastal livelihoods rebound and \
               change following destructive inundation events than livelihood rehabilitation aid. \
                Additionally, our case study suggests the human impact of coastal hazards can be felt outside \
                 the physical extent of inundation."
)

'of east asia water resources for the united kingdom population based on maternal mortality and flood protection in germany spain the united states usa from tanzania the united states transfer initiative of india 2016 national surveys from the case study of nepal river basin of india 2016 case study of nepal storm and case studies of nepal earthquake protection program in england and south africa data from south africa wide island coast based on local case studies of nepal storm region level data from the european states management case definition of nepal storm area level data from south africa data from the european union'

In [ ]:
summaries = []
for d in document[:20]:
  summaries.append([summarize(d)])

In [ ]:
summaries

[['promotes the presence of extracellular vesicles the liver injury and cancer cell cycle of their breast cancer in mice and the elderly regulation of the mtor pathway based cancer cell lines in ageing mice a family planning a family planning network meta analysis of law and cancer patient with oecd cancer patient with oecd cancer patient with oecd modified cells cross sectional features of web based therapies meta analyses of law cancer patient with rise regulation strategy in rise mechanisms structural liver cancer patient with law cancer patient with complete deficiency german society new zealand doped cells independent cancer stem cell cycle arrest view'],
 ['of green eu care products based on the green synthesis of synthetic biology solutions of pseudomonas aeruginosa evolutionary tools used by use of resources structural analysis by d based ligand stable isotopes in virtual screening based ligand structure and machine learning methods and machine learning strategy for drug source

In [ ]:
from collections import Counter
import math
from nltk.translate.bleu_score import sentence_bleu


## Implementation of BLEU

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
reference = summary[0]
candidate = summaries[0]
candidate_1 = ' '.join(candidate)
score = sentence_bleu(reference, candidate_1)
print(score)

7.778565201456134e-232


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
scores = []
for i in range(20):
    reference = summary[i]
    candidate = summaries[i]
    candidate_1 = " ".join(candidate)
    score = sentence_bleu(reference, candidate_1)
    scores.append(score)
print("Average Blue scores for 20 summaries", np.mean(scores))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

Average Blue scores for 20 summaries 7.48492284824238e-232


## Implementation of ROUGE

In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(candidate_1, reference)
print(scores)

{'rouge1': Score(precision=0.3888888888888889, recall=0.06796116504854369, fmeasure=0.11570247933884296), 'rougeL': Score(precision=0.2777777777777778, recall=0.04854368932038835, fmeasure=0.08264462809917356)}


In [ ]:
scores = {}
for i in range(20):
    reference = summary[i]
    candidate = summaries[i]
    candidate_1 = " ".join(candidate)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores_rouge = scorer.score(candidate_1, reference)
    #score_fuzz = fuzz.partial_ratio(reference,candidate_1)
    #score = sentence_bleu(reference, candidate_1)
    scores['rouge1'].append(scores_rouge)
    #scores.append(scores_rouge)
print("Average scores_rouge for 20 summaries", scores['rouge1'])

KeyError: ignored

### Implementation of Match Ratio Score


In [ ]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
print(fuzz.ratio(reference,candidate_1))
#91
print(fuzz.partial_ratio(reference,candidate_1)) ## best
#100

1
30


In [ ]:
scores = []
for i in range(20):
    reference = summary[i]
    candidate = summaries[i]
    candidate_1 = " ".join(candidate)
    score_fuzz = fuzz.partial_ratio(reference,candidate_1)
    #score = sentence_bleu(reference, candidate_1)
    scores.append(score_fuzz)
print("Average Fuzz scores for 20 summaries", np.mean(scores))

Average Fuzz scores for 20 summaries 33.25
